<a href="https://colab.research.google.com/github/spectr747/AFML/blob/master/22022025_telegramBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
import time

In [6]:
URL="https://auto.ru/-/ajax/desktop-search/listing/"
PARAMS = {
    "year_from": 2020,
    "section": "used",
    "category": "cars",
    "top_days": 1,
    "outup_type": "list",
    "geo_id": []
}
DICT_HEADER = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0",
    "Cookie": 'los=1; bltsr=1; coockoos=17; suid=4ef50d3323dfe1105d58b5b6626f8c1b.978f2c2be3372f4c587815056e386da0; _csrf_token=6629a1c87cf9e2438a8253586803c7b5d5699ac6d03d4584; autoru_sid=a%3Ag66c71d842jff020r01rgsgto0vj9ccb.9209de52a951e6dab18e59ef38695cc5%7C1724325252912.604800.ChDYYj3NDBvRcxxlbK0Lyw.mUBEBehLUd8JgCBtBKNELjRUHR1H31Am5BuGYj2DQu8; autoruuid=g66c71d842jff020r01rgsgto0vj9ccb.9209de52a951e6dab18e59ef38695cc5; from_lifetime=1724327960663; from=direct; autoru_sso_blocked=1; _yasc=IzZI+bpBKzRa5fBEr1eXSdL8xs40ZDgG+PqPRMt/kRRRZIn94ZymkStRoxFxTncazA==; Session_id=noauth:1724325253; sessar=1.1193.CiBCUPG4QlqRXnDfRV-mmZbMqBLfyur9PKw5M6otvrAYvQ.SN3PTwfsoBcVulD8erAhFI_k5MSQkPCwJL-LWWSRokk; yandex_login=; ys=c_chck.4038300478; i=qls+0YheKrw9axsSSUBgnfhg/Ne6EVe60yPWy6l9NC3mn85Pvkx4o4c09Dx9hm9U3Ui6PVvFePH8JCUw0QaXhWSkoiI=; yandexuid=9653613051724325253; mda2_beacon=1724325253121; sso_status=sso.passport.yandex.ru:synchronized_no_beacon; autoru_sso_redirect_blocked=1; layout-config={"screen_height":1080,"screen_width":1920,"win_width":1903,"win_height":549}; count-visits=3; yaPassportTryAutologin=1; _ym_uid=1724325257599081200; _ym_d=1724325259; _ym_isad=2; crookie=jexbSdHDls+k9DRzYc1r42dotg3mSFy1V/apmvJeV9fbCQs/OqdCwE0PeQABMwhnNaJCLv/Zls895SoAnrqpFfv+Fqw=; cmtchd=MTcyNDMyNTI1NzA0Mw==; fp=787b645d27b0c5fee17d6d05558d5258%7C1724325257167; gdpr=0',
    "x-csrf-token": "6629a1c87cf9e2438a8253586803c7b5d5699ac6d03d4584",
    "Origin": "https://auto.ru",
}

In [14]:
response = requests.post(URL, json=PARAMS, headers=DICT_HEADER)
total_page_count = response.json()['pagination']['total_page_count']

In [22]:
results = []
for i in tqdm(range(5)):
  PARAMS['page']= int(i)
  response = requests.post(URL, json=PARAMS, headers=DICT_HEADER)
  res = response.json()
  results.append(res['offers'])

results_chain = list(itertools.chain(*results))

100%|██████████| 5/5 [00:08<00:00,  1.73s/it]


In [33]:
auto_list = []
for page in range(len(results_chain)):
  if 'from' in list(list(results_chain[page]['predicted_price_ranges']['autoru'].keys())):
    auto_list.append(
        {
            "mark_code": results_chain[page]['vehicle_info']['mark_info']['code'],
            "model_code":  results_chain[page]['vehicle_info']['model_info']['code'],
            "year": results_chain[page]['documents']['year'],
            "price": results_chain[page]['price_info']['price'],
            "price_from": results_chain[page]['predicted_price_ranges']['autoru']['from'],
            "price_to": results_chain[page]['predicted_price_ranges']['autoru']['to'],
            "region_info": results_chain[page]['seller']['location']['region_info']['name'],
            "created_date": results_chain[page]['additional_info']['creation_date'],
            "update_date": results_chain[page]['additional_info']['update_date'],
            "saleId":results_chain[page]['saleId']

        }
    )
  else:
    {
            "mark_code": results_chain[page]['vehicle_info']['mark_info']['code'],
            "model_code":  results_chain[page]['vehicle_info']['model_info']['code'],
            "year": results_chain[page]['documents']['year'],
            "price": results_chain[page]['price_info']['price'],
            "price_from": 0,
            "price_to": 0,
            "region_info": results_chain[page]['seller']['location']['region_info']['name'],
            "created_date": results_chain[page]['additional_info']['creation_date'],
            "update_date": results_chain[page]['additional_info']['update_date'],
            "saleId":results_chain[page]['saleId']

        }

df = pd.DataFrame(auto_list)

In [34]:
df_filtered = df.copy()

In [41]:
df_filtered['percent'] = ((df_filtered['price_from'] - df['price']) / df['price_from']) * 100

In [42]:
df_filtered['percent'] = (df_filtered['percent']
  .replace([np.inf, -np.inf], np.nan)
  .fillna(0)
  .astype(int))

In [50]:
for date_col in ['created_date', 'update_date']:
  df_filtered[date_col] = (
      pd.to_datetime(df_filtered[date_col], unit='ms').dt.strftime("%Y-%m-%d %H:%M:%S")
  )

In [52]:
df_filtered['model_code'] = df_filtered['model_code'].str.casefold()
df_filtered['mark_code'] = df_filtered['mark_code'].str.casefold()

In [56]:
df_filtered['link'] = df_filtered.apply(
    lambda row: "https://auto.ru/cars/used/sale/{mark_code}/{model_code}/{saleid}".format(
        mark_code=row['mark_code'],
        model_code=row['model_code'],
        saleid=row['saleId'],
    ),
    axis=1,
)

In [60]:
df_filtered = df_filtered.sort_values(by='percent', ascending=False)
df_filtered.reset_index(drop=True, inplace=True)

In [61]:
df_filtered

,mark_code,model_code,year,price,price_from,price_to,region_info,created_date,update_date,saleId,percent,link
0,vaz,vesta,2020,630000,841000,975000,Россошь,2025-02-22 04:35:38,2025-02-22 04:35:38,1127150712-3775c3f4,25,https://auto.ru/cars/used/sale/vaz/vesta/11271...
1,vaz,vesta,2020,630000,841000,975000,Россошь,2025-02-22 04:35:38,2025-02-22 04:35:38,1127150712-3775c3f4,25,https://auto.ru/cars/used/sale/vaz/vesta/11271...
2,toyota,rav_4,2021,3050000,3819000,4336000,Ставрополь,2025-02-21 11:04:17,2025-02-22 05:51:01,1126981175-fa000aeb,20,https://auto.ru/cars/used/sale/toyota/rav_4/11...
3,bmw,5er,2022,4660000,5632000,6460000,Краснодар,2025-02-21 14:01:52,2025-02-06 06:38:02,1126934924-95361abc,17,https://auto.ru/cars/used/sale/bmw/5er/1126934...
4,honda,odyssey_na,2020,2670000,3241000,3655000,Тюмень,2025-02-21 12:01:06,2025-02-22 06:12:21,1125525384-09bc2e6f,17,https://auto.ru/cars/used/sale/honda/odyssey_n...
...,...,...,...,...,...,...,...,...,...,...,...,...
96,vaz,granta,2023,950000,692000,822000,Москва,2025-02-22 05:50:32,2025-02-22 05:50:32,1127151200-2278b843,-37,https://auto.ru/cars/used/sale/vaz/granta/1127...
97,ferrari,f8_tributo,2021,35994000,26165000,29979000,Краснодар,2025-02-21 09:30:41,2025-02-22 04:47:29,1126971135-d07440f9,-37,https://auto.ru/cars/used/sale/ferrari/f8_trib...
98,vaz,niva_legend,2024,1250000,894000,1026000,Самара,2025-02-22 05:34:20,2025-02-22 05:39:19,1126235078-12d058e9,-39,https://auto.ru/cars/used/sale/vaz/niva_legend...
99,vaz,granta,2023,1100000,742000,848000,Альшеевский район,2025-02-22 06:13:04,2025-02-22 06:13:04,1127151335-792b4894,-48,https://auto.ru/cars/used/sale/vaz/granta/1127...
